# HW2 - Scraping X and Creating TimeMaps
### Quentin Phillips
### DATA 440 Fall 2023
### 10/26/23

# 1. Collect URIs from tweets

## Below is the modified code that I used to scrape the base URLs. These URLs still required processing, which I used a separate function for which I will include later. I saved this as a python file in VScode and then ran it through windows powershell via >python3 "C:\\Users\\quill\\Downloads\\scrape_twitter_v2.py"

##I had to run this code several times to get many links. I attempted to scrape 1000 tweets per keyword, and the keywords I used included: Taylor Swift, Breaking News, Gaza, Halloween, etc.
## This process took longer than I anticipated, and in the future I would run it more times to collect well over 1000 links so that after cleaning and removing some links I still have sufficient numbers.

---



---



In [ ]:
import sys
import time

from bs4 import BeautifulSoup
from datetime import datetime
from NwalaTextUtils.textutils import genericErrorInfo
from NwalaTextUtils.textutils import getLinks

from playwright.sync_api import sync_playwright
from urllib.parse import quote_plus

from util import paral_rehydrate_tweets
from util import rehydrate_tweet
from util import write_tweets_to_jsonl_file

from scrape_twitter import get_auth_twitter_pg
from scrape_twitter import get_search_tweets
#from scrape_twitter import get_timeline_tweets
from util import write_tweets_to_jsonl_file

import gzip
import json


def is_twitter_user_auth(links, cur_page_uri):

    if( cur_page_uri.strip().startswith('https://twitter.com/home') ):
        return True

    logged_in_links = ['https://twitter.com/home', 'https://t.co/']

    for l in links:
        for log_l in logged_in_links:
            if( l['link'].startswith(log_l) ):
                return True
    return False

def scroll_up(page):
    page.evaluate("window.scrollTo( {'top': 0, 'left': 0, 'behavior': 'smooth'} );")

def scroll_down(page):
    page.evaluate("window.scrollTo( {'top': document.body.scrollHeight, 'left': 0, 'behavior': 'smooth'} );")

def post_tweet(page, msg, button_name='Post', after_post_sleep=2.5):
    #Post, Reply
    eval_str = f''' document.querySelectorAll('[aria-label$="{button_name}"]')[0].click(); '''
    page.evaluate(eval_str)
    time.sleep(1)
    page.keyboard.type(msg, delay=20)
    page.evaluate(''' document.querySelectorAll('[data-testid="tweetButton"]')[0].click(); ''')
    time.sleep(after_post_sleep)


def color_tweet(page, tweet_link):

    query_slc = f'''article = document.querySelectorAll('[href="{tweet_link}"]');'''
    page.evaluate(query_slc + '''
        if( article.length != 0 )
        {
            article = article[0];
            article.style.backgroundColor = 'red';
            i = 0;
            while(i < 1000)
            {
                if( article.nodeName == 'ARTICLE' )
                {
                    article.style.outline = "thick solid red";
                    article.className = "cust-tweet";
                    break;
                }
                article = article.parentElement;
                i++;
            }
        }
    ''')


def get_tweet_ids_user_timeline_page(screen_name, page, max_tweets):

    prev_len = 0
    empty_result_count = 0

    tweet_links = set()
    break_flag = False

    while( True ):

        page_html = page.content()
        soup = BeautifulSoup(page_html, 'html.parser')
        articles = soup.find_all('article')

        for i in range(len(articles)):

            t = articles[i]
            is_retweet = t.find('span', {'data-testid': 'socialContext'})
            is_retweet = False if is_retweet is None else is_retweet.text.strip().lower().endswith(' retweeted')

            tweet_datetime = ''
            tweet_link = t.find('time')

            if( tweet_link is None ):
                tweet_link = ''
            else:
                tweet_datetime = tweet_link.get('datetime', '')
                tweet_link = tweet_link.parent.get('href', '')

            if( tweet_link == '' ):
                continue


            if( screen_name != '' and is_retweet is False and tweet_link.startswith(f'/{screen_name}/') is False ):
                #This tweet was authored by someone else, NOT the owner of the timeline, and since it was not retweeted
                continue

            #color_tweet(page, tweet_link)
            tweet_links.add( tweet_link )
            twt_id = tweet_link.split('/status/')[-1]
            tweet_obj = rehydrate_tweet(twt_id)
            if( len(tweet_obj) == 0 ):
                print('\tOOPS! rehydration failed! patch rehydrate_tweet()')
                continue

            print( '\ttweets {}, datetime: {}, is_retweet: {}'.format(len(tweet_links), tweet_datetime, is_retweet) )
            #print( f'\tdo stuff here with tweet_obj: {tweet_obj.keys()}\n' )


            #file_path = "C:\\Users\\quill\\Downloads\\linksfile.txt"
            if 'urls' in tweet_obj.get('entities', []):
                for link in tweet_obj['entities']['urls']:
                    print(link['expanded_url'])
                    if 'twitter.com' not in link['expanded_url'] and 'youtu.be' not in link['expanded_url'] and 'soundcloud.com' not in link['expanded_url'] and 'twitch.com' not in link['expanded_url']and 'youtube.com' not in link['expanded_url']:
                        with open("C:\\Users\\quill\\Downloads\\linksfile.txt", 'a') as file:
                            file.writelines(link['expanded_url'] + '\n')

            if( len(tweet_links) == max_tweets ):
                print(f'exiting reached ({len(tweet_links)}) maximum: {max_tweets}')
                sys.exit(0)

        empty_result_count = empty_result_count + 1 if prev_len == len(tweet_links) else 0
        if( empty_result_count > 5 ):
            print(f'No new tweets found, so exiting')
            sys.exit(0)

        prev_len = len(tweet_links)
        print('\tthrottling/scrolling, then sleeping for 2 second\n')
        scroll_down(page)
        time.sleep(2)


def get_timeline_tweets(browser_dets, screen_name, max_tweets=20):

    screen_name = screen_name.strip()
    if( max_tweets < 0  or len(browser_dets) == 0 or screen_name == '' ):
        return {}

    print( f'\nget_timeline_tweets(): {screen_name}' )
    uri = f'https://twitter.com/{screen_name}/with_replies'

    payload = {'self': uri, 'tweets': []}
    browser_dets['page'].goto(uri)

    tweet_ids = get_tweet_ids_user_timeline_page( screen_name, browser_dets['page'], max_tweets )
    payload['tweets'] = paral_rehydrate_tweets(tweet_ids)

    return payload

def stream_tweets(browser_dets, query, max_tweets=20):

    query = query.strip()
    if( max_tweets < 0  or len(browser_dets) == 0 or query == '' ):
        return {}

    print('\nstream_tweets():')
    uri = 'https://twitter.com/search?q=' + quote_plus(query) + '&f=live&src=typd'

    payload = {'self': uri, 'tweets': []}
    browser_dets['page'].goto(uri)

    get_tweet_ids_user_timeline_page( '', browser_dets['page'], max_tweets )

def get_auth_twitter_pg(playwright, callback_uri=''):

    print('\nget_auth_twitter_pg()')

    chromium = playwright.firefox #"chromium" or "firefox" or "webkit".
    browser = chromium.launch(headless=False)
    context = browser.new_context()
    page = context.new_page()

    sleep_seconds = 3
    page.goto('https://twitter.com/login')

    while( True ):

        print(f'\twaiting for login, sleeping for {sleep_seconds} seconds')

        time.sleep(sleep_seconds)
        page_html = page.content()
        page_links = getLinks(uri='', html=page_html, fromMainTextFlag=False)
        scroll_down(page)

        if( is_twitter_user_auth(page_links, page.url) ):

            print('\tauthenticated')
            if( callback_uri != '' ):
                page.goto(callback_uri)
                print(f'\tauthenticated, loaded {callback_uri}')

            print('\tsleeping for 3 seconds')
            time.sleep(3)
            return {
                'page': page,
                'context': context,
                'browser': browser
            }

    return {}

def main():

    if( len(sys.argv) != 3 ):
        print(f'Usage example:\n\tpython {sys.argv[0]} "williamsburg" 20')
        return

    with sync_playwright() as playwright:

        browser_dets = get_auth_twitter_pg(playwright)
        if( len(browser_dets) == 0 ):
            return

        stream_tweets(browser_dets, sys.argv[1], max_tweets=int(sys.argv[2]))

if __name__ == "__main__":

    main()




## This code returned a .txt file with many links that included redirects or youtube links. To remove the youtube links, I added the "'youtube.com' not in link['expanded_url']:" and "'youtu.be' not in link['expanded_url']: lines to the code to exclude these video links.

##Next I ran the following python script in PowerShell to return the processed URIs file. I ensured that the saved links returned a HTTP 200 status code to make sure they are not redirecting.

##I ran this with >Python3 "C:\\Users\\quill\\Downloads\\URI_Retriever.py"

In [ ]:
import requests

def resolve_urls(url_list, timeout=10):
    resolved_urls = set()

    for url in url_list:
        try:
            response = requests.head(url, allow_redirects=True, timeout=timeout)
            if response.status_code == 200:
                resolved_url = response.url
                resolved_urls.add(resolved_url)
            else:
                print(f"Skipping non-200 URL: {url}")
        except requests.exceptions.Timeout:
            print(f"Skipping timed out URL: {url}")
        except requests.exceptions.RequestException:
            print(f"Failed to connect: {url}")

    return resolved_urls

def read_urls(input_file_path):
    unique_urls = set()

    with open(input_file_path, "r") as input_file:
        for line in input_file:
            url = line.strip()
            unique_urls.add(url)

    return list(unique_urls)

input_file_path = "C:\\Users\\quill\\Downloads\\linksfile.txt"
output_file_path = "C:\\Users\\quill\\Downloads\\resolvedURLS.txt"

unique_urls = read_urls(input_file_path)
resolved_urls = resolve_urls(unique_urls, timeout=10)

# Save the resolved URLs to resolvedURLS.txt
with open(output_file_path, "w") as output_file:
    for url in resolved_urls:
        output_file.write(url + "\n")

print(f"Resolved URLs saved to {output_file_path}")

##This returned the resolvedURLS.txt file that I then used through MemGator to create the TimeMaps. I have included the .txt file in this repo.

# 2. Generate TimeMaps
## I used Python to automate running the links through MemGator. The script I used is included below with some notes about switching to Linux:

In [ ]:
import subprocess

file_path = "C:\\Users\\quill\\Downloads\\resolvedURLs.txt"
#Changed the binary path when running on Linux
memgator_binary = "C:\\Users\\quill\\Downloads\\memgator-windows-amd64.exe"

with open(file_path, "r") as uri_file:
    for line in uri_file:
        uri = line.strip()

        # Ended up not using WSL because I had to switch to Linux machine, but still including because this is intended for PowerShell
        cmd = ["wsl", memgator_binary, "--format=JSON", uri]

        try:
            subprocess.run(cmd, check=True)
            print(f"TimeMap Generated for: {uri}")
        except subprocess.CalledProcessError as e:
            # Error handling
            print(f"Error processing URI: {uri}")
            print(f"Error: {e}")

print("All TimeMaps Generated.")


# 3. Memento Table

|Mementos|URI-Rs|
|:---|:---|
|0|0|
|1-5|0|
|6-10|0|
|11-15|0|
|16-20|0|

#Extra Credit

https://conifer.rhizome.org/qdphillips/volleyball-history

Q1. Volleyball is a big part of my life but it is a relatively small sport. I wanted to get a sense of how well documented professional results are and do my part to archive them.

Q2. Archiving these webpages was actually a very smooth experience using conifer through google chrome.

Q3. The archived pages look very similar to the original, even with the complex formatting.

Q4. The archive saved 100 URLs. Way more than I was anticipating.

![pic](https://github.com/QuentinPhil/DATA44002/blob/main/HW2/Screenshot 2023-10-26 154510.png?raw=true)

#References
###(Many not directly relevant, but I needed to debug/learn about linux and command line)

* Insert Reference 1, <https://www.geeksforgeeks.org/reading-and-writing-json-to-a-file-in-python/>
* Insert Reference 2, <https://www.annasyme.com/docs/python-subprocess.html>
* Insert Reference 3, <https://askubuntu.com/questions/588390/how-do-i-check-whether-a-module-is-installed-in-python-and-install-it-if-needed>
* Insert Reference 4, <https://askubuntu.com/questions/1309941/what-does-a-no-such-file-or-directory-mean-and-how-do-i-correct-the-problem/reallyreallyreally-extra-long-URI/>
* Insert Reference 5, <https://unix.stackexchange.com/questions/166728/chmod-cannot-access-file-no-such-file-or-directory-error-when-the-file-exis>
* Insert Reference 6, <https://learn.microsoft.com/en-us/windows/wsl/install-manual#step-6---install-your-linux-distribution-of-choice>
* Insert Reference 7, <https://chat.openai.com/>
* Insert Reference 8, <https://learn.microsoft.com/en-us/windows/wsl/basic-commands#unregister-or-uninstall-a-linux-distribution>
* Insert Reference 9, <https://requests.readthedocs.io/en/latest/>
* Insert Reference 10, <https://stackoverflow.com/questions/25651990/oserror-winerror-193-1-is-not-a-valid-win32-application>
* Insert Reference 11, <https://opensource.com/article/22/7/use-bash-automate-tasks-linux>
* Insert Reference 12, <https://stackoverflow.com/questions/4278083/how-to-get-request-uri-without-context-path>
* Insert Reference 13, <https://stackoverflow.com/questions/44327037/read-links-from-a-list-from-a-txt-file-python>
* Insert Reference 14, <https://www.geeksforgeeks.org/get-method-python-requests/>
* Insert Reference 15, <https://stackoverflow.com/questions/46783078/uri-encoding-in-python-requests-package>
* Insert Reference 16, <https://unix.stackexchange.com/questions/238180/execute-shell-commands-in-python>
* Insert Reference 17, <https://realpython.com/python-subprocess/>
* Insert Reference 18, <https://www.tutorialspoint.com/python/python_command_line_arguments.htm>

>